In [1]:
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import time
import pandas as pd
# own code
import libarary.graph_tools as gtl
import libarary.invarients as inva
import libarary.cpp as cpp

In [2]:
def test(G: nx.Graph):
    clocks = []
    start = time()
    dists = gtl.convert_dist_dict_to_narray(
        dict(nx.all_pairs_dijkstra_path_length(G)), list(G.nodes)
    )
    end = time()
    clocks.append(end - start)
    print("clock convert: ", clocks[-1])
    start = time()
    q_extend3 = cpp.q_extend_single(dists, 3)
    end = time()
    clocks.append(end - start)
    print("clock q_extend3: ", clocks[-1], q_extend3)
    start = time()
    excess = cpp.excess_global_single(dists)
    end = time()
    clocks.append(end - start)
    print("clock excess_global: ", clocks[-1], excess)
    start = time()
    q_packing3 = cpp.q_packing_single(dists, 3)
    end = time()
    clocks.append(end - start)
    print("clock q_packing: ", clocks[-1], q_packing3)

    res = {
        "|V|": G.number_of_nodes(),
        "q-extend3": q_extend3,
        "excess-global": excess,
        "q-packing3": q_packing3,
    }
    return res


res = {}


# mouse brain

In [3]:
G = gtl.read_graph_from_edgelist("networks-random/brain274/bn-mouse_brain_1.edges")
print("number of nodes: ", G.number_of_nodes())

clocks = []
start = time()
dists = gtl.convert_dist_dict_to_narray(
    dict(nx.all_pairs_dijkstra_path_length(G)), list(G.nodes)
)
end = time()
clocks.append(end - start)
print("clock convert: ", clocks[-1])
start = time()
q_extend3 = cpp.q_extend_single(dists, 3)
end = time()
clocks.append(end - start)
print("clock q_extend3 single: ", clocks[-1], q_extend3)


number of nodes:  213
clock convert:  0.5511531829833984
clock q_extend3 single:  0.8245596885681152 0.0001771636105943839


In [4]:

start = time()
q_extend3 = cpp.q_extend_multi(dists, 3)
end = time()
clocks.append(end - start)
print("clock q_extend3 multi: ", clocks[-1], q_extend3)

clock q_extend3 multi:  0.05719137191772461 0.0001771636105943839


# aves social

In [5]:
g1 = gtl.read_graph_from_edgelist(
    "networks-random/aves-wildbirds/aves-wildbird-network.edges"
)
print(g1.number_of_nodes())
res["aves"] = test(g1)

clocks = []
start = time()

end = time()
clocks.append(end - start)
print("clock convert: ", clocks[-1])



202
clock convert:  0.16045832633972168
clock q_extend3:  0.6924483776092529 0.0003940692576720359
clock excess_global:  0.26055407524108887 7.0
clock q_packing:  0.7035162448883057 0.5
clock convert:  0.5446383953094482
clock q_extend3 multi:  0.052374839782714844 0.0001771636105943839


# Mammalia social connections

In [7]:
G = gtl.read_graph_from_edgelist(
    "networks-random/memalia-big/mammalia-voles-bhp-trapping.edges"
)
print("number of nodes: ", G.number_of_nodes())
start = time()
dists = gtl.convert_dist_dict_to_narray(
    dict(nx.all_pairs_dijkstra_path_length(G)), list(G.nodes)
)
q_extend3 = cpp.q_extend_multi(dists, 3)
end = time()
clocks.append(end - start)
print("clock q_extend3 multi: ", end-start, q_extend3)
# res["Mammalia social"] = test(g2)


number of nodes:  1686
clock q_extend3 multi:  29.003262758255005 2.1119993241602164e-05


# Power Eris

In [8]:
mm = sio.mmread("networks1/power/power-eris1176.mtx")
g3: nx.Graph = nx.from_scipy_sparse_array(mm)

print("number of nodes: ", g3.number_of_nodes())

# res["power-eris"] = test(g3)


number of nodes:  1176


In [10]:
G = g3
start = time()
dists = gtl.convert_dist_dict_to_narray(
    dict(nx.all_pairs_dijkstra_path_length(G)), list(G.nodes)
)
q_extend3 = cpp.q_extend_multi(dists, 5)
end = time()
clocks.append(end - start)
print("clock q_extend3 multi: ", end-start, q_extend3)

# Results

In [8]:
ds = pd.DataFrame(res)
ds.to_csv("results/first-results.csv")
ds

,mouse-brain,aves,Mammalia social,power-eris
|V|,213.000000,202.000000,1686.000000,1176.000000
q-extend3,0.000177,0.000394,0.000021,0.000153
excess-global,3.000000,7.000000,28.000000,105.000000
q-packing3,1.000000,0.500000,0.500000,1.000000
